In [ ]:
import gluonbook  as gb
from mxnet import gluon,init,nd
from mxnet.gluon import data as gdata ,loss as gloss,model_zoo
from mxnet.gluon import utils as gutils
import os 
import zipfile
import mxnet as mx

## Dataset

In [ ]:
if not os.path.exists('../data/hotdog.zip'):
    os.system('curl https://apache-mxnet.s3-accelerate.amazonaws.com/gluon/dataset/hotdog.zip -o ../data/hotdog.zip')
with zipfile.ZipFile('../data/hotdog.zip') as zin:
    zin.extractall('../data/')
train_data=gdata.vision.ImageFolderDataset('../data/hotdog/train/')
test_data=gdata.vision.ImageFolderDataset('../data/hotdog/test/')

In [ ]:
train_augs=gdata.vision.transforms.Compose([gdata.vision.transforms.RandomResizedCrop(224),
                                          gdata.vision.transforms.RandomFlipLeftRight(),
                                          gdata.vision.transforms.ToTensor(),
                                          gdata.vision.transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

test_augs=gdata.vision.transforms.Compose([gdata.vision.transforms.Resize(256),
                                         gdata.vision.transforms.CenterCrop(224),
                                         gdata.vision.transforms.ToTensor(),
                                         gdata.vision.transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])



In [ ]:
batch_size=128
train_iter=gdata.DataLoader(train_data.transform_first(train_augs),batch_size=batch_size,shuffle=True)
test_iter=gdata.DataLoader(test_data.transform_first(test_augs),batch_size=batch_size)

## model(fine tuning)

In [ ]:
ctx=mx.gpu()
pretrained_net=model_zoo.vision.resnet18_v2(pretrained=True)
finetune_net=model_zoo.vision.resnet18_v2(classes=2)
finetune_net.output.initialize(force_reinit=True,init=init.Xavier())
finetune_net.features=pretrained_net.features
finetune_net.collect_params().reset_ctx(ctx)

## train & eva

In [ ]:
num_epochs=5
loss=gloss.SoftmaxCrossEntropyLoss()
trainer=gluon.Trainer(finetune_net.collect_params(),'sgd',{'learning_rate':0.01,'wd':0.001})
gb.train(train_iter,test_iter,finetune_net,loss,trainer,ctx,num_epochs) 